<a href="https://colab.research.google.com/github/pinballsurgeon/deluxo_adjacency/blob/main/auto_quantum_circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Quantum Circuits
#### dehls - pinballsurgeon@gmail.com

In [1]:
!pip install --quiet tensorflow_quantum

In [2]:
import cirq
import sympy
import tensorflow
import numpy
import gc

%matplotlib inline
import matplotlib.pyplot

from cirq.contrib.svg import SVGCircuit

import tensorflow_quantum

Helper Functions

In [3]:
# given a data and out qubit structure, build dynamic circuit
class circuit_struct():
    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout
    
    def add_layer(self, circuit, gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(gate(qubit, self.readout)**symbol)

In [4]:
# given a combination of qubit grid size and gate population, build 
def create_quantum_model(gridSize, hyperGates):

    # cirq
    data_qubits = cirq.GridQubit.rect(gridSize, gridSize)
    readout = cirq.GridQubit(-1, -1)         
    circuit = cirq.Circuit()
    
    # prepare the readout qubit
    circuit.append(cirq.X(readout))
    circuit.append(cirq.H(readout))
    
    # create qubit
    builder = circuit_struct(
        data_qubits = data_qubits,
        readout=readout)

    # buid layers
    for gate in hyperGates:

      # builder.add_layer(circuit, cirq.XX, "xx1")
      builder.add_layer(circuit, gate, "zz1")
   
    # prepare the readout qubit
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)

In [ ]:
### pixel to qubit
def convert_to_circuit(image):

    values = numpy.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
        if value:
            circuit.append(cirq.X(qubits[i]))
    return circuit

In [5]:
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., numpy.newaxis]/255.0, x_test[..., numpy.newaxis]/255.0

In [6]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

In [7]:
x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

In [8]:
x_train_small = tensorflow.image.resize(x_train, (4,4)).numpy()
x_test_small = tensorflow.image.resize(x_test, (4,4)).numpy()

In [9]:
x_train_nocon, y_train_nocon = x_train_small[:100], y_train[:100]

In [11]:
x_train_circ = [convert_to_circuit(x) for x in x_train_nocon]
x_test_circ = [convert_to_circuit(x) for x in x_test_small]

In [12]:
x_train_tfcirc = tensorflow_quantum.convert_to_tensor(x_train_circ)
x_test_tfcirc = tensorflow_quantum.convert_to_tensor(x_test_circ)

In [13]:
EPOCHS = 3
BATCH_SIZE = 32

NUM_EXAMPLES = len(x_train_tfcirc)

In [14]:
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_hinge_sub = y_train_nocon[:NUM_EXAMPLES]

In [18]:
class mem_release(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [32]:
experiment_count = 0
experiment_log = {}
for gates in [[cirq.XX, cirq.ZZ], [cirq.CZ, cirq.ZZ]]:
  for gridSize_int in [1, 2, 3]:

    model_circuit, model_readout = create_quantum_model(gridSize_int, gates)
    
    # Build the Keras model.
    model = tensorflow.keras.Sequential([
        # The input is the data-circuit, encoded as a tf.string
        tensorflow.keras.layers.Input(shape=(), dtype=tensorflow.string),
        # tensorflow.keras.layers.Dense(64, activation='relu'),
        # The PQC layer returns the expected value of the readout gate, range [-1,1].
        tensorflow_quantum.layers.PQC(model_circuit, model_readout),])

    model.compile(
      loss=tensorflow.keras.losses.mse,
      optimizer=tensorflow.keras.optimizers.Adam(),
      metrics=[tensorflow.keras.metrics.BinaryAccuracy()]
      )
    
    print(model.summary())

    qnn_history = model.fit(
        x_train_tfcirc_sub, y_train_hinge_sub,
        batch_size=32,
        epochs=EPOCHS,
        verbose=1,
        callbacks=[mem_release()]
        )

    qnn_results = model.evaluate(x_test_tfcirc, y_test)

    experiment_count += 1
    experiment_log[experiment_count] = {'circuit' : model_circuit
                                       ,'history' : qnn_history}
    

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pqc_24 (PQC)                (None, 1)                 1         
                                                                 
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
4/4 [==============================] - 3s 295ms/step - loss: 1.0134 - binary_accuracy: 0.5500
Epoch 2/3
4/4 [==============================] - 1s 220ms/step - loss: 0.8462 - binary_accuracy: 0.6000
Epoch 3/3
62/62 [==============================] - 2s 17ms/step - loss: 0.9145 - binary_accuracy: 0.5645
Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pqc_25 (PQC)                (None, 1)                 4         
                                                

In [34]:
SVGCircuit(experiment_log[3]['circuit'])
print(experiment_log[3]['history'])
# SVGCircuit(model_circuit)